In [ ]:
colab = False

import torch
import os 

if colab:
  print("PyTorch has version {}".format(torch.__version__))

  # Install torch geometric
  if 'IS_GRADESCOPE_ENV' not in os.environ:
    !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-geometric

  !git clone https://github.com/thibautvalour/Graph-Diffusion-Convolution.git
  %cd Graph-Diffusion-Convolution

import numpy as np
import networkx as nx
import pandas as pd
import math as math
from torch_geometric.datasets import Planetoid
from torch.nn.functional import nll_loss

from matrix_format import gdc_pagerank, gdc_heat, compute_Lsym, compute_Lrw
from models import GCN_Classifier
from utils import train, test

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

# 1 Cora

In [ ]:
dataset = Planetoid(root='../cora', name='Cora')
data = dataset[0]

train_idx = data.train_mask.to(device)
val_idx = data.val_mask.to(device)
test_idx = data.test_mask.to(device)

Gnx = nx.from_pandas_edgelist(pd.DataFrame(data['edge_index'].T,
                                           columns=['source', 'target']))
Gnx = Gnx.to_undirected()

# Exctract adjacency matrix
A = nx.adjacency_matrix(Gnx).toarray()

### 1.1 Laplacian matrix

In [ ]:
Lsym = compute_Lsym(A)
Lsym = torch.from_numpy(Lsym).float().to(device)

In [ ]:
args = {
    'device': device,
    'hidden_layers': 1,
    'hidden_dim': 264,
    'dropout': 0.5,
    'lr': 3e-4,
    'epochs': 3,
    'trans_matrix': Lsym
}

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

In [ ]:
Lrw = compute_Lrw(A)
Lrw = torch.from_numpy(Lrw).float().to(device)
args['trans_matrix'] = Lrw

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

### 1.2 Page rank

In [ ]:
pagerank = gdc_pagerank(A, 0.05, 1e-4)
pagerank = torch.from_numpy(pagerank).float().to(device)

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')

### 1.3 Heat

In [ ]:
heat = gdc_heat(A, 3, 10, 1e-4)
heat = torch.from_numpy(heat).float().to(device)
args['trans_matrix'] = heat

In [ ]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       hidden_layers=args['hidden_layers'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = nll_loss

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  train_acc, test_acc = test(model, data, train_idx, test_idx, args['trans_matrix'])

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100*train_acc:.2f}%, '
        f'Test: {100*test_acc:.2f}%')